In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import datetime
import random
from selenium import webdriver
import time
from selenium.common.exceptions import TimeoutException


In [21]:
base_url = 'https://www.worldweatheronline.com/ikorodu-weather-history/lagos/ng.aspx'
date = datetime.datetime(2024, 8, 31)
weather_df = pd.DataFrame()

In [6]:
def get_next_date(date=date, days = 1) :
    offset = datetime.timedelta(days=days)
    return date - offset

In [27]:
driver = webdriver.Chrome()
driver.get(base_url)

In [18]:
def get_data():
    html = driver.page_source
    df_list = pd.read_html(StringIO(html))

    df =  df_list[0]
    df.columns = df.iloc[0]

    time = df['Time'].str[:2]
    temperature = df['Time'].str[5:]

    df['Time'] = time
    df['Temperature'] = temperature

    df = df[['Time', 'Temperature', 'Forecast', 'Rain', 'Rain %', 'Cloud', 'Pressure', 'Wind', 'Gust']][2:].reset_index(drop=True)

    soup = BeautifulSoup(html)
    table = soup.select('table.days-details-table')[0]

    arrows = table.select('svg.days-arrow-white')
    
    directions = [r.attrs['style'] for r in arrows]
    df['Direction (deg)'] = directions
    df.rename(columns={'Time': 'Time (hr)', 'Temperature': 'Temperature (°c)', 'Forecast': 'Forecast (°c)', 'Rain': 'Rain (mm)', 'Rain %': 'Rain (%)', 'Cloud': 'Cloud (%)', 'Pressure': 'Pressure (mb)', 'Wind': 'Wind (km/h)', 'Gust': 'Gust (km/h)'}, inplace=True)

    day_information_div = soup.select('div.days-inform')[0]
    info = day_information_div.select('span')
    extra_information = []


    for i in info:
        extra_information.append(i.text)

    df['Moonrise (hr)'] = extra_information[0]
    df['Moonset (hr)'] = extra_information[1]
    df['Sunrise (hr)'] = extra_information[2]
    df['Sunset (hr)'] = extra_information[3]
    df['Weather'] = soup.select('div.days-collapse-forecast')[0].text

    return df

In [8]:
def search(date = get_next_date().strftime("%d-%m-%Y")) :
    timeout = random.randint(5, 10)    
    dateInput = driver.find_element('xpath', '//input[@type="date"]')
    dateInput.send_keys(date)

    button = driver.find_element('xpath', '//input[@type="submit"]')
    button.click()

    time.sleep(timeout)
    df = get_data()
    df['Date'] = date

    return df

In [28]:
flag = True

while flag:
    current_date = date.strftime("%d-%m-%Y")
    try:
        df = search(current_date)
    except ValueError:
        print('ValueError')
        time.sleep(30)
    except TimeoutException:
        print('TimeoutException')
        time.sleep(30)

    weather_df = pd.concat([weather_df, df], ignore_index=True)
    date = get_next_date(date)
    l = len(weather_df)
    flag = l/8 != (365*8)
    print(l)

weather_df

4416
4424
TimeoutException
4432
4440
4448
4456
4464
4472
4480
4488
4496
4504
4512
4520
4528
4536
4544
4552
4560
4568
4576
4584
4592
4600
4608
4616
4624
4632
4640
4648
4656
4664
4672
4680
4688
4696
4704
4712
4720
4728
4736
4744
4752
4760
4768
4776
4784
4792
4800
4808
4816
4824
4832
4840
4848
4856
4864
4872
4880
4888
4896
4904
4912
4920
4928
4936
4944
4952
4960
4968
4976
4984
4992
5000
5008
5016
5024
5032
5040
5048
5056
5064
5072
5080
5088
5096
5104
5112
5120
5128
5136
5144
5152
5160
5168
5176
5184
5192
5200
5208
5216
5224
5232
5240
5248
5256
5264
5272
5280
5288
5296
5304
5312
5320
5328
5336
5344
5352
5360
5368
5376
5384
5392
5400
5408
5416
5424
5432
5440
5448
5456
5464
5472
5480
5488
5496
5504
5512
5520
5528
5536
5544
5552
5560
5568
5576
5584
5592
5600
5608
5616
5624
5632
5640
5648
5656
5664
5672
5680
5688
5696
5704
5712
5720
5728
5736
5744
5752
5760
5768
5776
5784
5792
5800
5808
5816
5824
5832
5840
5848
5856
5864
5872
5880
5888
5896
5904
5912
5920
5928
5936
5944
5952
5960
5968
5976
598

,Time (hr),Temperature (°c),Forecast (°c),Rain (mm),Rain (%),Cloud (%),Pressure (mb),Wind (km/h),Gust (km/h),Direction (deg),Moonrise (hr),Moonset (hr),Sunrise (hr),Sunset (hr),Weather,Date
0,00,25 °c,27 °c,0.1 mm,45%,87%,1013 mb,18 km/h,27 km/h,transform: rotate(225.3deg);,04:31 AM,05:17 PM,06:39 AM,06:53 PM,Light rain shower,31-08-2024
1,03,24 °c,26 °c,0.4 mm,100%,100%,1012 mb,19 km/h,28 km/h,transform: rotate(225.1deg);,04:31 AM,05:17 PM,06:39 AM,06:53 PM,Light rain shower,31-08-2024
2,06,24 °c,26 °c,0.3 mm,100%,100%,1013 mb,18 km/h,27 km/h,transform: rotate(230.1deg);,04:31 AM,05:17 PM,06:39 AM,06:53 PM,Light rain shower,31-08-2024
3,09,25 °c,28 °c,0.1 mm,45%,76%,1014 mb,21 km/h,28 km/h,transform: rotate(226.8deg);,04:31 AM,05:17 PM,06:39 AM,06:53 PM,Light rain shower,31-08-2024
4,12,27 °c,29 °c,0.0 mm,45%,61%,1014 mb,21 km/h,27 km/h,transform: rotate(229.2deg);,04:31 AM,05:17 PM,06:39 AM,06:53 PM,Light rain shower,31-08-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23355,09,26 °c,29 °c,1.7 mm,100%,75%,1013 mb,9 km/h,19 km/h,transform: rotate(234.1deg);,08:09 AM,08:29 PM,06:38 AM,06:52 PM,Moderate or heavy rain shower,03-09-2016
23356,12,27 °c,30 °c,2.5 mm,100%,86%,1012 mb,14 km/h,20 km/h,transform: rotate(192.8deg);,08:09 AM,08:29 PM,06:38 AM,06:52 PM,Moderate or heavy rain shower,03-09-2016
23357,15,28 °c,32 °c,0.6 mm,100%,71%,1010 mb,19 km/h,27 km/h,transform: rotate(200deg);,08:09 AM,08:29 PM,06:38 AM,06:52 PM,Moderate or heavy rain shower,03-09-2016
23358,18,26 °c,29 °c,0.0 mm,0%,21%,1011 mb,16 km/h,28 km/h,transform: rotate(219.9deg);,08:09 AM,08:29 PM,06:38 AM,06:52 PM,Moderate or heavy rain shower,03-09-2016


In [29]:
weather_df.to_csv('weather.csv')